In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, division, print_function
import sys, imp
imp.reload(sys)
#sys.setdefaultencoding('utf8')
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
import codecs

import random
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn import metrics

dataset = {}
labels = set()
featurenames = {}

#rt -> r
with codecs.open("lebeled_data.txt", "r", encoding="utf-8") as infile:
    for line_index, line in enumerate(infile):
        if line.strip() == '':
            continue
        try:
            pmid, original, classname, features = line.split("\t")
            pmid = int(float(pmid))
            features = features.strip().split(" ")
            #print(pmid, classname, features)
            labels.add(classname)
            if not pmid in dataset:
                dataset[pmid] = {}

            # create features as dicts (featurename = 1.0) for DictVectorizer later on
            dataset[pmid][classname] = {}
            for feature in features:
                dataset[pmid][classname][feature] = 1.0
                featurenames[feature] = 1.0
        except Exception as e:
            print("Malformed '%s' (#%s) --> %s" % (line, line_index + 1, e))

#print("-----",dataset)
unmatched = 0
for pmid in dataset:
    data = dataset[pmid]
    prevfeatures = None
    for classname in data:
        features = data[classname].keys()
        if not prevfeatures is None:
            if not features == prevfeatures:
                print(pmid, classname, features == prevfeatures)
        prevfeatures = features


# determine which classifiers we need to train based on the labels
classifiers = {}
for label in labels:
    labelprefix = label.split("-")[0]
    if not labelprefix in classifiers:
        classifiers[labelprefix] = set()
    classifiers[labelprefix].add(label)
#print(classifiers)

# create learning algorithm instances to try on the data
try_algorithms = {}

try_algorithms['SVC'] = svm.SVC(kernel='linear', C=1, random_state=0)
try_algorithms['NB'] = MultinomialNB()
try_algorithms['LReg'] = LogisticRegression()
try_algorithms['SGD'] = SGDClassifier()

for classifier, labels in classifiers.items():
    print("training classifier", classifier, "for labels", ", ".join(labels))
    # gather data with the labels in this classifier
    classifier_labels = []
    classifier_data = []
    for pmid in dataset:
        for label in dataset[pmid]:
            if not label in labels:
                # not relevant for current classifier
                continue
            features = dataset[pmid][label]
            classifier_labels.append(label)
            classifier_data.append(features)
    random.shuffle(classifier_data)

    # encode features
    # (create integer indices for all feature names)
    dv = DictVectorizer(sparse=False)
    classifier_data_transformed = dv.fit_transform(classifier_data)
    #print("-----",len(classifier_data), ": ",classifier_data)
    # encode labels
    # (create integer encodings for class labels)
    le = LabelEncoder()
    le.fit(list(labels))
    classifier_labels_transformed = le.fit_transform(classifier_labels)
    #print("-----", len(classifier_labels), ": ",classifier_labels)

    # the _transformed arrays now contain only numeric information

    # now we can train and evaluate a classifier (in 5-fold cross validation)
    for algo, clf in try_algorithms.items():
        print("training", algo, "on", classifier)
        scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'accuracy']
        scores = cross_validate(clf, classifier_data_transformed, classifier_labels_transformed, scoring=scoring, cv=5, return_train_score=False)

        # output scores and timing information:
        for k in sorted(scores.keys()):
            print("\t%s %s %s: %0.2f (+/- %0.2f)" % (classifier, algo, k, scores[k].mean(), scores[k].std() * 2))



Malformed '12422056	Moving forward: Herceptin in the adjuvant setting.
' (#233) --> not enough values to unpack (expected 4, got 2)
Malformed 'HER2 overexpression/amplification, which is an early event in breast cancer development, is associated with a poor prognosis and may predict response to therapy.	CGE-increased	subj_adjuvant subj_setting subj_ subj_HER2 subj_overexpression subj_/ subj_amplification obj_/ obj_amplification obj_event obj_breast cancer obj_development obj_poor obj_prognosis
' (#234) --> not enough values to unpack (expected 4, got 3)
Malformed '12422056	Moving forward: Herceptin in the adjuvant setting.
' (#235) --> not enough values to unpack (expected 4, got 2)
Malformed 'HER2 overexpression/amplification, which is an early event in breast cancer development, is associated with a poor prognosis and may predict response to therapy.	IGE-unchanged	subj_adjuvant subj_setting subj_ subj_HER2 subj_overexpression subj_/ subj_amplification obj_/ obj_amplification obj_even

c:\program files\python36\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


	IGE SVC fit_time: 0.43 (+/- 0.12)
	IGE SVC score_time: 0.28 (+/- 0.08)
	IGE SVC test_accuracy: 0.32 (+/- 0.12)
	IGE SVC test_f1_macro: 0.27 (+/- 0.12)
	IGE SVC test_precision_macro: 0.28 (+/- 0.14)
	IGE SVC test_recall_macro: 0.28 (+/- 0.11)
training NB on IGE
	IGE NB fit_time: 0.01 (+/- 0.00)
	IGE NB score_time: 0.00 (+/- 0.00)
	IGE NB test_accuracy: 0.31 (+/- 0.09)
	IGE NB test_f1_macro: 0.27 (+/- 0.11)
	IGE NB test_precision_macro: 0.27 (+/- 0.12)
	IGE NB test_recall_macro: 0.28 (+/- 0.10)
training LReg on IGE
	IGE LReg fit_time: 0.01 (+/- 0.00)
	IGE LReg score_time: 0.00 (+/- 0.00)
	IGE LReg test_accuracy: 0.34 (+/- 0.12)
	IGE LReg test_f1_macro: 0.28 (+/- 0.14)
	IGE LReg test_precision_macro: 0.29 (+/- 0.16)
	IGE LReg test_recall_macro: 0.29 (+/- 0.13)
training SGD on IGE


c:\program files\python36\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\program files\python36\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_gr

	IGE SGD fit_time: 0.03 (+/- 0.01)
	IGE SGD score_time: 0.00 (+/- 0.00)
	IGE SGD test_accuracy: 0.34 (+/- 0.13)
	IGE SGD test_f1_macro: 0.28 (+/- 0.15)
	IGE SGD test_precision_macro: 0.28 (+/- 0.17)
	IGE SGD test_recall_macro: 0.29 (+/- 0.14)
training classifier CCS for labels CCS-normalTOcancer, CCS-unidentifiable, CCS-cancerTOnormal
training SVC on CCS
	CCS SVC fit_time: 0.37 (+/- 0.02)
	CCS SVC score_time: 0.26 (+/- 0.02)
	CCS SVC test_accuracy: 0.43 (+/- 0.08)
	CCS SVC test_f1_macro: 0.35 (+/- 0.09)
	CCS SVC test_precision_macro: 0.36 (+/- 0.09)
	CCS SVC test_recall_macro: 0.35 (+/- 0.08)
training NB on CCS
	CCS NB fit_time: 0.01 (+/- 0.00)
	CCS NB score_time: 0.00 (+/- 0.00)
	CCS NB test_accuracy: 0.44 (+/- 0.06)
	CCS NB test_f1_macro: 0.35 (+/- 0.08)
	CCS NB test_precision_macro: 0.36 (+/- 0.10)
	CCS NB test_recall_macro: 0.36 (+/- 0.08)
training LReg on CCS
	CCS LReg fit_time: 0.01 (+/- 0.00)
	CCS LReg score_time: 0.00 (+/- 0.00)
	CCS LReg test_accuracy: 0.45 (+/- 0.05)
	CCS LRe

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

	CCS SGD fit_time: 0.02 (+/- 0.00)
	CCS SGD score_time: 0.00 (+/- 0.00)
	CCS SGD test_accuracy: 0.40 (+/- 0.04)
	CCS SGD test_f1_macro: 0.33 (+/- 0.10)
	CCS SGD test_precision_macro: 0.34 (+/- 0.12)
	CCS SGD test_recall_macro: 0.34 (+/- 0.08)
training classifier PT for labels PT-causality, PT-, PT-observation
training SVC on PT
	PT SVC fit_time: 0.39 (+/- 0.08)
	PT SVC score_time: 0.28 (+/- 0.07)
	PT SVC test_accuracy: 0.42 (+/- 0.06)
	PT SVC test_f1_macro: 0.37 (+/- 0.10)
	PT SVC test_precision_macro: 0.38 (+/- 0.11)
	PT SVC test_recall_macro: 0.38 (+/- 0.08)
training NB on PT
	PT NB fit_time: 0.01 (+/- 0.00)
	PT NB score_time: 0.00 (+/- 0.00)
	PT NB test_accuracy: 0.41 (+/- 0.09)
	PT NB test_f1_macro: 0.37 (+/- 0.11)
	PT NB test_precision_macro: 0.37 (+/- 0.11)
	PT NB test_recall_macro: 0.37 (+/- 0.10)
training LReg on PT
	PT LReg fit_time: 0.01 (+/- 0.00)
	PT LReg score_time: 0.00 (+/- 0.00)
	PT LReg test_accuracy: 0.41 (+/- 0.05)
	PT LReg test_f1_macro: 0.34 (+/- 0.04)
	PT LReg tes

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

	PT SGD fit_time: 0.02 (+/- 0.00)
	PT SGD score_time: 0.00 (+/- 0.00)
	PT SGD test_accuracy: 0.40 (+/- 0.08)
	PT SGD test_f1_macro: 0.36 (+/- 0.10)
	PT SGD test_precision_macro: 0.37 (+/- 0.14)
	PT SGD test_recall_macro: 0.36 (+/- 0.09)
training classifier CGE for labels CGE-decreased, CGE-increased
training SVC on CGE
	CGE SVC fit_time: 0.28 (+/- 0.05)
	CGE SVC score_time: 0.29 (+/- 0.10)
	CGE SVC test_accuracy: 0.61 (+/- 0.07)
	CGE SVC test_f1_macro: 0.51 (+/- 0.08)
	CGE SVC test_precision_macro: 0.53 (+/- 0.09)
	CGE SVC test_recall_macro: 0.52 (+/- 0.07)
training NB on CGE
	CGE NB fit_time: 0.01 (+/- 0.00)
	CGE NB score_time: 0.00 (+/- 0.00)
	CGE NB test_accuracy: 0.59 (+/- 0.13)
	CGE NB test_f1_macro: 0.52 (+/- 0.15)
	CGE NB test_precision_macro: 0.53 (+/- 0.16)
	CGE NB test_recall_macro: 0.52 (+/- 0.14)
training LReg on CGE
	CGE LReg fit_time: 0.01 (+/- 0.00)
	CGE LReg score_time: 0.00 (+/- 0.00)
	CGE LReg test_accuracy: 0.63 (+/- 0.12)
	CGE LReg test_f1_macro: 0.49 (+/- 0.11)
	CG

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\